# Intro to the object cell project!

## Wrangling

First, let's wrangle some data from Nagelhus: https://search.kg.ebrains.eu/?category=Dataset&q=moser%20rat#0162668e-bef0-4754-8562-40440db5bc0c


I've saved it in my downloads folder. You can open it like so:

In [ ]:
from scipy.io import loadmat
data = loadmat("/home/nolanlab/Downloads/singlecellanalyses_CA1.mat")

Here's a function which reads in a "trial" from this data, and makes a "session" thy way I'd like it (not too bad, wrangling-wise):

In [ ]:
import pynapple as nap
import numpy as np
from object_scores.core import SingleSession, SessionGroup

def session_from_trial(trial):

    try: 
        object_position = trial[3][0]['x'][0][0][0], trial[3][0]['y'][0][0][0]
    except:
        object_position = None

    if object_position is not None:
        # a dictionary of objects in each trial. We'll name this object "A". The code package will keep track of which objects appear in which sessions.
        object_dict = {"A": object_position}
    else:
        object_dict = {}

    positions = trial[1]
    x = np.transpose(positions['x'][0][0])[0]
    y = np.transpose(positions['y'][0][0])[0]
    timestamps = np.transpose(positions['timestamp'][0][0])[0]

    Px = nap.Tsd(timestamps, x)
    Py = nap.Tsd(timestamps, y)

    spikes_np = [np.transpose(spike_train[0])[0] for spike_train in trial[4][0]]

    spikes = nap.TsGroup(spikes_np)

    return SingleSession(spikes, Px, Py, object_dict)

Now take three trials from one day, and make three "sessions" of them by using this newly written function

In [ ]:
session_index = 0
rat_25224 = data['dataset'][0]['sessions'][0][0][session_index]

no_object_trial = rat_25224[2][0][0]
object_trial = rat_25224[2][0][1]
moved_trial = rat_25224[2][0][2]

no_object_session = session_from_trial(no_object_trial)
object_session = session_from_trial(object_trial)
moved_session = session_from_trial(moved_trial)

nagelhus_day = SessionGroup({'no-object': no_object_session, 'object': object_session, 'moved': moved_session})

Ok - that's all the wrangling. Now we compute scores. Each score, e.g. the Object Responsiveness Index (ori) score, has a, e.g. `compute_ori` method and a `plot_ori` method. These methods have a simple structure: they check if they have the appropriate sessions to compute the score, then grab all data needed to compute the score from the `SessionGroup` object, then compute. Let's compute stuff:

In [ ]:
nagelhus_day.compute_information_content()
nagelhus_day.compute_ori()
nagelhus_day.compute_increase()

As you compute the scores, they get saved in your GroupSessions object (our one is `nagelhus_day`):

In [ ]:
nagelhus_day.scores

I've tried to write plotting functions which demonstrate the basics of the score. You need to pass a `cluster_id`. Let's plot the `increase` visualisation. This score computes the firing rate near an object in a non-object trial and then an object trial, and computes how much firing changed. Then does this lots of times for shuffled data and tries to see if the increase is significant.

In [ ]:
fig = nagelhus_day.plot_increase(cluster_id=2)

This plot makes us doubt the "normal" approximation we make for the score distribution... nice!